In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss
from tqdm.notebook import tqdm

args = get_args()
args.arch = 'resnet50'
args.dataset = 'CIFAR10'
args.selection_method = 'none'

# Example Usage

model, target_layers = load_architecture(args)
model.to('cuda')

args.epochs = 2
args.pruning_ratio = 0.5
args.delta = 1
args.batch_size = 2
args.pruning_strategy = 'TS_pruning'
args.batch_strategy = 'random'
args.sample_size= 5

# train_dataset = IndexedDataset()
print('init weighted dataset')
train_dataset = WeightedDataset(args, train=True, prune_ratio = args.pruning_ratio,  )

# for i in range(len(train_dataset)):
#     print(train_dataset[i])
import numpy as np

def obtain_latent_dataset(model, dataset, batch_size=32):

    # Assuming the dataset is a list or similar iterable with a known length
    num_samples = len(dataset)

    # Assume the dimensionality of the latent representation can be determined from one sample
    image,label, idx = dataset[0]
    image = torch.Tensor(image).to('cuda').unsqueeze(0)
    print(image.shape)
    first_latent_rep = model.get_latent_representation(image)
    latent_dim = first_latent_rep.shape[1]
    print(first_latent_rep.shape)
        
    # Preallocate the array for the latent representations
    latent_dataset = torch.zeros((num_samples, latent_dim))


    for i in tqdm( range(0, num_samples, batch_size) ):
        # Get the current batch of data
        batch_indices = list(range(i, min(i + batch_size, num_samples)))
        images,labels,idxs = dataset[batch_indices]
        images = images.to('cuda')
            
        # Process the batch to get latent representations
        batch_latent_reps = model.get_latent_representation(images) 
            
        # Store the results in the preallocated array
        latent_dataset[i:i + batch_size] = batch_latent_reps.detach().cpu()

    return latent_dataset

features =  obtain_latent_dataset(model,train_dataset,64)
train_dataset.define_latent_features(features)

train_sampler = DistributedCustomSampler(args, train_dataset, num_replicas=2, rank=1, drop_last=True)

print('init dataloder')
trainloader = DataLoader(train_dataset, batch_size=None, sampler = train_sampler,) 

./data
init weighted dataset
Files already downloaded and verified
load dataloader
torch.Size([1, 3, 32, 32])
torch.Size([1, 2048])


  0%|          | 0/782 [00:00<?, ?it/s]

25000 50000
init dataloder


In [2]:
from losses import get_loss, get_eval_loss

iterations = 2
rank = 'cuda'

optimizer = torch.optim.SGD( model.parameters(),lr=args.init_lr, weight_decay=args.weight_decay, momentum=args.momentum, nesterov=True, )

for iteration in range(iterations):

    model.train()
    train_sampler.set_epoch(iteration)

    for batch_id, batch in enumerate( trainloader ):

        data, target, idxs = batch

        data, target = data.to(rank), target.to(rank) 
         
        loss_values, clean_values, robust_values, logits_nat, logits_adv = get_loss(args, model, data, target, optimizer)

        train_dataset.update_scores(idxs, clean_values, robust_values, loss_values, logits_nat, logits_adv)
        loss = train_dataset.compute_loss(idxs, loss_values)

pruning
remove tail


TypeError: dot(): argument 'tensor' (position 1) must be Tensor, not numpy.ndarray